In [11]:

import torch
SEED = 123
torch.manual_seed(SEED) 

import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
# from torchsummary import summary
from torchinfo import summary
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import warnings
from tqdm import tqdm
from datetime import datetime
warnings.filterwarnings('ignore')

from PIL import Image
import os
import glob

# check OS is Window or Mac
import platform
device = torch.device("cpu")

if platform.system() == 'Windows':
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
elif platform.system() == 'Darwin':
    try:
        device = torch.device("mps")
    except:
        device = torch.device("cpu")

float_formatter = "{:.2f}".format
np.set_printoptions(formatter={'float_kind': float_formatter})

print(device)

cpu


In [13]:

import torch
import torch.nn as nn

class CustomFCNetwork(nn.Module):
    def __init__(self):
        super(CustomFCNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(3 * 96 * 96, 100)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(100, 32)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(32, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

# Instantiate the model
model = CustomFCNetwork()

# Print the model architecture and number of trainable parameters
print(model)
print(f"Number of trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

CustomFCNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=27648, out_features=100, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=100, out_features=32, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=32, out_features=10, bias=True)
)
Number of trainable parameters: 2768462


In [14]:
model_uploaded = CustomFCNetwork()
model_name = "FCN"
model_uploaded.load_state_dict(torch.load(f"./models//best_model_{model_name}.pth"))

model_uploaded.to(device)

print(model_uploaded)

CustomFCNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=27648, out_features=100, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=100, out_features=32, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=32, out_features=10, bias=True)
)


In [15]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt


# Definir la ruta de la carpeta
path = "data_split/test/"

# Crear listas para almacenar imágenes y etiquetas
images = []
labels = []

# Iterar por las 10 clases
for clse in range(1, 11):
    # Obtener la ruta para cada clase
    ruta = os.path.join(path, str(clse))
    # Obtener los datos
    for archivo in os.listdir(ruta):
        if archivo.endswith(".png"):
            ruta_img = os.path.join(ruta, archivo)
            img = Image.open(ruta_img).convert("L")
            img_array = np.array(img)
            images.append(img_array)
            labels.append(clse)

# Convertir las listas de imágenes y etiquetas a tensores de PyTorch
images_tensor = torch.tensor(images, dtype=torch.float).view(-1, 1, 96, 96)
labels_tensor = torch.tensor(labels, dtype=torch.long)

# Crear un conjunto de datos PyTorch
dataset = torch.utils.data.TensorDataset(images_tensor, labels_tensor)


In [38]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torchvision import models
from torch.autograd import Variable
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

# Definir la función FGSM
def fgsm_attack(image, epsilon, data_grad):
    # Obtener el signo del gradiente
    sign_data_grad = data_grad.sign()
    # Crear la imagen adversarial usando el signo del gradiente
    perturbed_image = image + epsilon * sign_data_grad
    # Asegurarse de que la imagen resultante esté en el rango [0, 1]
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    return perturbed_image

# Función para cargar el modelo
def load_model(model_path):
    model = torch.load(model_path)
    model.eval()
    return model

# Función para cargar y preprocesar una imagen
def load_and_preprocess_image(image_path):
    transform = transforms.Compose([
        transforms.Resize((96, 96)),
        transforms.ToTensor(),
    ])
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0)
    return Variable(image, requires_grad=True)

# Función para realizar el ataque adversarial y mostrar las imágenes
def perform_fgsm_attack(model, image, epsilon_values):
    # Obtener la etiqueta verdadera de la imagen
    true_label = torch.argmax(model(image).data).item()

    # Inicializar la figura para mostrar las imágenes
    plt.figure(figsize=(10, len(epsilon_values) * 2))

    for i, epsilon in enumerate(epsilon_values):
        # Calcular el gradiente de la pérdida con respecto a la entrada
        loss = nn.CrossEntropyLoss()
        model.zero_grad()
        output = model(image)
        loss_value = loss(output, torch.tensor([true_label]))
        loss_value.backward()
        data_grad = image.grad.data

        # Realizar el ataque FGSM
        perturbed_image = fgsm_attack(image, epsilon, data_grad)

        # Mostrar la imagen perturbada
        perturbed_label = torch.argmax(model(perturbed_image).data).item()
        plt.subplot(len(epsilon_values), 2, 2 * i + 1)
        plt.imshow(perturbed_image.squeeze().detach().numpy().transpose(1, 2, 0))
        plt.title(f'FGSM Attack (Epsilon={epsilon})\nPredicted Label: {perturbed_label}')

        # Mostrar la diferencia entre la imagen original y la perturbada
        difference = (perturbed_image - image).squeeze().detach().numpy().transpose(1, 2, 0)
        plt.subplot(len(epsilon_values), 2, 2 * i + 2)
        plt.imshow(difference)
        plt.title('Difference')

    plt.tight_layout()
    plt.show()

# Ruta a las imágenes
path = 'data_split/test/'

# Cargar modelos
cnn_model = load_model('/home/enzoc/Escritorio/lab4ML/Training-Inference-Attacks-Synthesis-Robustness-/models/best_model_LeNet5_final.pth')
fc_model = load_model('/home/enzoc/Escritorio/lab4ML/Training-Inference-Attacks-Synthesis-Robustness-/models/best_model_FCN_final.pth')

# Lista de valores epsilon para variar la intensidad del ataque
epsilon_values = [0.01, 0.05, 0.1, 0.2, 0.3]

# Iterar sobre las clases y realizar el ataque adversarial
for clse in range(1, 11):
    # Obtener la ruta para cada clase
    ruta = os.path.join(path, str(clse))
    # Obtener los datos
    for archivo in os.listdir(ruta):
        if archivo.endswith(".png"):
            ruta_img = os.path.join(ruta, archivo)

            # Cargar y preprocesar la imagen
            image = load_and_preprocess_image(ruta_img)

            # Realizar el ataque adversarial en la CNN
            print(f'Class: {clse}, CNN Model:')
            perform_fgsm_attack(cnn_model, image, epsilon_values)

            # Realizar el ataque adversarial en el modelo totalmente conectado
            print(f'Class: {clse}, Fully Connected Model:')
            perform_fgsm_attack(fc_model, image, epsilon_values)


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.